# Aaron Szabo

Please replace first_name and last_name with your information. This notebook will be your template that you should follow for this project. Feel free to create any subsections within each section.

#### Imports

In [40]:
import numpy as np
import pandas as pd
import string

from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.moses import MosesDetokenizer

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, merge
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#### Constants

In [2]:
# Reproducibility
np.random.seed(1234)

MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.15

## Section 1: Load Data

There is no code for you to fill out in this section but please make sure you understand what the code is doing so you aren't confused in later parts. If you want to run this code on Colab, you can pass in the links to the CSV rather than the file path.

You can find the training and testing data here: https://www.kaggle.com/c/quora-question-pairs/data

#### File Paths

In [3]:
TRAIN_CSV = 'train.csv'
TEST_CSV = 'test.csv'
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'

In [4]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [5]:
train_df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [6]:
test_df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


## Section 2: Data Processing

In this section we will proccess our data into a format suitable for inputting into a LSTM or more specifically, a Siamese LSTM. The general structure we will follow is:

1. Preprocess the text (clean it up)
2. Tokenize every word in the text (replace each word with an index number)
3. Pad every sequence of indices with zeros to make them all the same length
4. Build an embedding matrix that we can use to look up every indices word embedding

In this section, you will be responsible for both preprocessing the text in the function `clean_text` and also building the embedding matrix. Fillers for where you should write your code are marked with #YOUR CODE HERE.

Rather than training our own word embeddings on the dataset, it is generally better to use a pretrained one since it is much more accurate. In our example, we will be using a Word2Vec model trained on Google News but you are free to use any you would like. Spacy's Glove model is a good choice.

You can download the pretrained model here: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing

#### Load Pretrained Word2Vec Model

We will load the model into a Keyed Vectors file. Using this, we can get the embedding of any word by calling `.word_vec(word)` and we can get all the words in the model's vocabulary through `.vocab`. It is important to note that if the word does not exist in the model's vocabulary, you can NOT get it's embedding and so standard practice is to ignore it or initialize it to a random or zero vector.

In [8]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True, limit=500000)
print('Done')

Done


### Preprocessing (TODO)

The text was processed in order to clean up the data. 
* puncuation marks are removed
* non-alphabetic 'words' are removed, e.g. "'s"
* stopwords are removed
* verbs are converted to their infinitive form

In [9]:
def clean_text(text):
    text = str(text)
    
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    tokens2 = [w.translate(table) for w in tokens]
    tokens3 = [w for w in tokens2 if w.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens4 = [w for w in tokens3 if not w in stop_words]
    porter = PorterStemmer()
    tokens5 = [porter.stem(w) for w in tokens4]
    detokenizer = MosesDetokenizer()
    text = detokenizer.detokenize(tokens5)
    
    # Return type should be str
    return text

Applying the preprocessing `clean_text` function to every element in the training and testing data.

In [11]:
X_train_1 = [clean_text(x) for x in train_df['question1']]
X_train_2 = [clean_text(x) for x in train_df['question2']]
labels = train_df['is_duplicate']
print('Loaded Training Data')

X_test_1 = [clean_text(x) for x in test_df['question1']]
X_test_2 = [clean_text(x) for x in test_df['question2']]
print('Loaded Testing Data')

Loaded Training Data
Loaded Testing Data


### Tokenizer

To avoid manually having to assign indices and filtering out unfrequent words, we can use a Tokenizer to do this for us. It essentially creates a map of every unique word and an assigned index to it. We specify a parameter called `num_words` which says to only care about the top 20000 most frequent words. 

In [12]:
MAX_NB_WORDS = 20000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_train_1 + X_train_2 + X_test_1 + X_test_2)
print('Finished Building Tokenizer')

Finished Building Tokenizer


Applying the tokenizer to the training and testing data.

In [13]:
train_sequences_1 = tokenizer.texts_to_sequences(X_train_1)
train_sequences_2 = tokenizer.texts_to_sequences(X_train_2)
print('Finished Tokenizing Training')

test_sequences_1 = tokenizer.texts_to_sequences(X_test_1)
test_sequences_2 = tokenizer.texts_to_sequences(X_test_2)
print('Finished Tokenizing Testing')

Finished Tokenizing Training
Finished Tokenizing Testing


Number of unique words in tokenizer. Has to be <= 20,000.

In [14]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 108604 unique tokens


Pad sequences all to the same length of 30 words.

In [15]:
train_data_1 = pad_sequences(train_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
train_data_2 = pad_sequences(train_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', train_data_1.shape)
print('Shape of label tensor:', labels.shape)
print('Finished Padding Training')

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
print('Finished Padding Testing')

Shape of data tensor: (404290, 30)
Shape of label tensor: (404290,)
Finished Padding Training
Finished Padding Testing


### Embedding Matrix (TODO)

The embedding matrix is a `n x m` matrix where `n` is the number of words and `m` is the dimension of the embedding. In our case, `m=300` and `n=20000`. We take the min between the number of unique words in our tokenizer and max words in case there are less unique words than the max we specified. 

Row `i` in the matrix should contain the embedding of the word with index `i` in the tokenizer. An easy way to create this would be to iterate over `word_index.items()` which gives you the word and it's index. Keep in mind that you can't generate an embedding for a word not in your word2vec model vocabulary.

In [17]:
nb_words = min(MAX_NB_WORDS, len(word_index))

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, index in word_index.items():
    if index<20000 and word in word2vec.vocab:
        embedding_matrix[index,:] = word2vec.word_vec(word)
print("Finished creating embedding matrix")

Finished creating embedding matrix


### Formatting Data

Here we just format the data into each half for the input (left and right). There is no code to write here but understand what it is doing.

In [46]:
# Random shuffle
perm = np.random.permutation(len(train_data_1))
idx_train = perm[:int(len(train_data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(train_data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = np.vstack((train_data_1[idx_train], train_data_2[idx_train]))
data_2_train = np.vstack((train_data_2[idx_train], train_data_1[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))
print('Finished Creating Training Data')

data_1_val = np.vstack((train_data_1[idx_val], train_data_2[idx_val]))
data_2_val = np.vstack((train_data_2[idx_val], train_data_1[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))
print('Finished Creating Validation Data')

Finished Creating Training Data
Finished Creating Validation Data


## Section 3: Building the Model

In this section you will write code to build the actual Siamese network. It should take in two arguments (question1 and question2) and then output a single number representing the probability that the two questions are duplicates.

The model should take in each input sentence, replace it with it's embeddings, then run the new embedding vector through a LSTM layer. The output of each LSTM layer should be concatenated together and then a standard Dense model can be used.

Make sure to note that you should only use one LSTM layer that is shared by both the left and the right half. 

Make sure to title your output layers as `predictions`.

### Build Model (TODO)

In [51]:
left_input = Input(shape=(30,), dtype='int32')
right_input = Input(shape=(30,), dtype='int32')
embedding_layer = Embedding(20000, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_NB_WORDS, trainable=False)
left_embedded = embedding_layer(left_input)
right_embedded = embedding_layer(right_input)
left_model = Dropout(0.2)(left_embedded)
right_model = Dropout(0.2)(right_embedded)
model = LSTM(100)
left_output = model(left_model)
right_output = model(right_model)
left_output = Dropout(0.2)(left_output)
right_output = Dropout(0.2)(right_output)
predictions = concatenate([left_output, right_output])
predictions = Dense(1,activation="sigmoid")(predictions)
print("Finished building model")

Finished building model


### Compiling Model

In [52]:
model = Model(inputs=[left_input, right_input], outputs=predictions)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

## Section 4: Training the Model

In this section we will simply train the model. We use the Early Stopping argument to end training if the loss or accuracy don't improve within 3 epochs.

Since the training time is incredibly long (30 minutes or so on a CPU), only train it for one epoch if you don't have time. For better results, train it to around 50 epochs.

In [57]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

hist = model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val), \
        epochs=10, batch_size=2048, shuffle=True, \
        callbacks=[early_stop])

Train on 687292 samples, validate on 121288 samples
Epoch 1/10
687292/687292 [==============================] - 520s 757us/step - loss: 0.5459 - acc: 0.7249 - val_loss: 0.5360 - val_acc: 0.7327
Epoch 2/10
687292/687292 [==============================] - 516s 751us/step - loss: 0.5275 - acc: 0.7373 - val_loss: 0.5232 - val_acc: 0.7427
Epoch 3/10
687292/687292 [==============================] - 516s 751us/step - loss: 0.5138 - acc: 0.7463 - val_loss: 0.5168 - val_acc: 0.7474
Epoch 4/10
687292/687292 [==============================] - 516s 750us/step - loss: 0.5019 - acc: 0.7538 - val_loss: 0.5165 - val_acc: 0.7490
Epoch 5/10
687292/687292 [==============================] - 515s 750us/step - loss: 0.4910 - acc: 0.7599 - val_loss: 0.5165 - val_acc: 0.7467
Epoch 6/10
687292/687292 [==============================] - 516s 751us/step - loss: 0.4811 - acc: 0.7657 - val_loss: 0.5036 - val_acc: 0.7584
Epoch 7/10
687292/687292 [==============================] - 516s 751us/step - loss: 0.4711 - acc

### Results

In [58]:
hist.history

{'acc': [0.7248956775574119,
  0.737347444789578,
  0.7463378011276666,
  0.7537713228529156,
  0.7598677127227382,
  0.7657153000235526,
  0.7710099928914236,
  0.7756310272638484,
  0.7795856782432378],
 'loss': [0.5459290541220257,
  0.5274997776135069,
  0.5138215278342549,
  0.5019437526635017,
  0.4909860180424219,
  0.48105141785025435,
  0.47110257770265135,
  0.46256982635025423,
  0.4546618327479964],
 'val_acc': [0.7326858386020926,
  0.7427033176955852,
  0.7473698965470431,
  0.7490188641320448,
  0.7467103093856636,
  0.758376756127061,
  0.7595887474834144,
  0.7619550161048437,
  0.7645438954309803],
 'val_loss': [0.5359862937193481,
  0.5231903300193501,
  0.5168004283984602,
  0.5164676649370276,
  0.5164755796601453,
  0.5035652156952904,
  0.5049841041953683,
  0.5044520871340147,
  0.5059474569029532]}

In [60]:
prev_hist2 = hist

In [59]:
prev_hist.history

{'acc': [0.6950262770065657],
 'loss': [0.5812028301110251],
 'val_acc': [0.715289228970202],
 'val_loss': [0.5610955431541379]}